In [10]:
import numpy as np
import pandas as pd 
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
from ipywidgets import HTML
from ipyleaflet import Popup, Map, Marker, LayersControl
from IPython import display

class desafioBiodiversidade():
    def __init__(self,filePath):
        try:
            f = open(filePath,"r")
            self.originalData = f.readlines()
            self.data = [l.rstrip().split(";") for l in self.originalData]
            f.close()

            self.AmeacaList = []
            self.plotMap = []
            self.dataAux = []
            self.dataT = []            
            
        except:
            print("Error: cannot open file.")
            data = None  
    
    #1
    #Faça um método que retorna a média de dados faltantes por coluna
    def retornaMediaDadosFaltantesPorColuna(self):
        data = self.data
        nlines = len(data)-1
        
        # lista de tamanho igual ao numero de colunas
        nblank = [0]*len(data[0])
        
        # cada posicao da lista armazena o contador de linhas faltantes da coluna correspondente
        for line in data:
            for ind,column in enumerate(line):
                if column == "Sem Informações" or column == "":
                    nblank[ind]+=1
        
        # calcula a media dos dados faltantes de cada coluna
        nblank = [np.round((x/nlines)*100,2) for x in nblank]
        
        # passa a lista para um dicionario
        # para impressao utilizando DataFrame do Pandas
        dict = {'Titulo': data[0], 'Value': nblank}
        df = pd.DataFrame(dict)
        print(df.to_string(index=False))
        
        
    #2
    #Para cada item identifique até qual nível taxônomico a ocorrência foi identificada.
    def retornaNivelTaxonomicoCadaOcorrencia(self):
        lista = self.data

        filo = []
        for i in range(len(lista[0])):
            
            if lista[0][i] == 'Especie':
              
                for k in range(1,len(lista)):
                    
                    if lista[k][i] != 'Sem Informações':
                        filo.append('{} - Especie'.format(k))
                    
                    elif lista[k][i-1] != 'Sem Informações':
                        filo.append('{} - Genero'.format(k))
                        
                    elif lista[k][i-2] != 'Sem Informações':
                        filo.append('{} - Familia'.format(k))
                        
                    elif lista[k][i-3] != 'Sem Informações':
                        filo.append('{} - Ordem'.format(k))
                    
                    elif lista[k][i-4] != 'Sem Informações':
                        filo.append('{} - Classe'.format(k))
                        
                    elif lista[k][i-5] != 'Sem Informações':
                        filo.append('{} - Filo'.format(k))
                        
                    else:
                        filo.append('{} - Reino'.format(k))
     
        return filo  
    
    #3
    #Monte filtros de ocorrências por estados, nome de espécie (nome exato ou parte do nome) e 
    #categoria de ameaça, e outros filtros que julgar relevante.
    def retonaSeOcorrenciaExiste(self, filterAmeaca):

        # Lê cada coluna por vez        
        for j in range(len(self.data[0])):        
            for i in range(len(self.data)-1):
                self.dataAux.append(self.data[i][j])
                # Cria uma nova "matriz" transposta com os dados
            self.dataT.append(self.dataAux)
            self.dataAux = []
        
        
        ##### Filtros de busca
        # Filtro de ameaça
        for i in range(len(self.dataT)):
            if self.dataT[i][0] == "Estado de conservacao":
                for j in range(len(self.dataT[i])):
                    if self.dataT[i][j] == filterAmeaca :
                        self.plotMap.append([self.dataT[i+7][j],self.dataT[i+8][j], self.dataT[i+5][j]])
                        self.AmeacaList.append(["Amostra {}: {}: ".format(j, self.dataT[i-12][j]), filterAmeaca])
                    
        return self.AmeacaList
        
    #4
    #Crie uma funcionalidade para avaliar se a informação de longitude e latitude 
    #corresponde a informação presente na localização
    def metodoPlotMap(self):
        vectorInformacoes = self.plotMap
        key = '0573d875005f45bebe452ff81c26b684'
        geocoder = OpenCageGeocode(key)

        i = 0
        center = (0.0, 0.0)
        itemMapa = Map(center = center, zoom = 3, close_popup_on_click = False)
        itemMapa.add_control(LayersControl())
        for item in vectorInformacoes:
            #if i <= 15:
                center = (item[0], item[1])
                marker = Marker(location = center, draggable = False)
                teste = HTML()
                teste.value = "<b>" + item[2] + "</b>"
                popup = Popup(
                    location = center,
                    child = teste,
                    close_button = True,
                    auto_close = False,
                    close_on_escape_key = False
                )
                itemMapa.add_layer(marker)
                itemMapa.add_layer(popup)
            #i+=1
        return itemMapa
    
filePath1 = "portalbio_export_16-10-2019-14-39-54.csv"
filePath2 = "portalbio_export_17-10-2019-13-06-22.csv"
filePath3 = "portalbio_export_17-10-2019-13-15-00.csv"

filePath7Daniel1 = "07-PR-testeCsv.csv"
filePath7Daniel2 = "07-SP-testeCsv.csv"

In [17]:
#PARTE GABRIELA
biodiversidade = desafioBiodiversidade(filePath7Daniel2)

In [18]:
biodiversidade.retornaMediaDadosFaltantesPorColuna()
#PARTE GABRIELA

                              Titulo   Value
                 Nome da instituicao  100.00
                Sigla da instituicao    0.00
               Nome da base de dados    0.00
              Sigla da base de dados    0.00
           Responsavel pelo registro    0.00
        Numero do registro no portal    0.00
 Numero do registro na base de dados    2.57
                    Data do registro    0.00
                      Data do evento    0.00
                    Data de Carencia    0.00
                     Nome cientifico    5.54
                          Nome comum   64.59
    Nome cientifico na base de dados    0.00
                    Nivel taxonomico    5.54
                Numero de individuos    0.00
                               Reino  100.00
                                Filo    5.54
                              Classe    5.54
                               Ordem    5.54
                             Familia    6.08
                              Genero    6.62
          

In [19]:
#PARTE BRUNA
print(biodiversidade.retornaNivelTaxonomicoCadaOcorrencia())
#PARTE BRUNA

['1 - Genero', '2 - Especie', '3 - Especie', '4 - Especie', '5 - Especie', '6 - Especie', '7 - Especie', '8 - Especie', '9 - Familia', '10 - Especie', '11 - Especie', '12 - Reino', '13 - Especie', '14 - Especie', '15 - Especie', '16 - Especie', '17 - Especie', '18 - Especie', '19 - Especie', '20 - Especie', '21 - Especie', '22 - Especie', '23 - Especie', '24 - Especie', '25 - Especie', '26 - Especie', '27 - Especie', '28 - Especie', '29 - Especie', '30 - Especie', '31 - Especie', '32 - Especie', '33 - Especie', '34 - Especie', '35 - Especie', '36 - Especie', '37 - Especie', '38 - Especie', '39 - Especie', '40 - Especie', '41 - Especie', '42 - Especie', '43 - Especie', '44 - Especie', '45 - Especie', '46 - Especie', '47 - Ordem', '48 - Familia', '49 - Reino', '50 - Especie', '51 - Especie', '52 - Especie', '53 - Especie', '54 - Especie', '55 - Especie', '56 - Especie', '57 - Especie', '58 - Especie', '59 - Especie', '60 - Especie', '61 - Especie', '62 - Especie', '63 - Especie', '64 - E

In [20]:
#PARTE RAFAEL
print(biodiversidade.retonaSeOcorrenciaExiste("Espécie Ameaçada"))
#PARTE RAFAEL

[['Amostra 49: Sem Informações: ', 'Espécie Ameaçada'], ['Amostra 53: Pseudotocinclus tietensis: ', 'Espécie Ameaçada'], ['Amostra 111: Taunayia bifasciata: ', 'Espécie Ameaçada'], ['Amostra 156: Conopophaga lineata: ', 'Espécie Ameaçada'], ['Amostra 214: Sem Informações: ', 'Espécie Ameaçada'], ['Amostra 245: Thamnophilus caerulescens: ', 'Espécie Ameaçada'], ['Amostra 246: Thamnophilus caerulescens: ', 'Espécie Ameaçada'], ['Amostra 250: Sem Informações: ', 'Espécie Ameaçada'], ['Amostra 291: Acianthera: ', 'Espécie Ameaçada'], ['Amostra 292: Dryadella: ', 'Espécie Ameaçada'], ['Amostra 317: Genidens barbus: ', 'Espécie Ameaçada'], ['Amostra 340: Genidens barbus: ', 'Espécie Ameaçada'], ['Amostra 369: Genidens barbus: ', 'Espécie Ameaçada'], ['Amostra 393: Genidens barbus: ', 'Espécie Ameaçada'], ['Amostra 409: Genidens barbus: ', 'Espécie Ameaçada'], ['Amostra 433: Sem Informações: ', 'Espécie Ameaçada'], ['Amostra 434: Sem Informações: ', 'Espécie Ameaçada'], ['Amostra 442: Sem Inf

In [21]:
#PARTE RAFAEL + DANIEL
map = biodiversidade.metodoPlotMap()
map
#PARTE RAFAEL + DANIEL

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …